## Sundance - Dataset

In [170]:
# #Import Libraries
import pandas as pd

import glob
from collections import defaultdict
from datetime import datetime

In [171]:
dict_weather_id_filepath = {}
for var_file in (glob.glob("sundance/SunDance_data_release/weather/*")):
#     print(var_file.split(".")[0].split("_")[-1])
    dict_weather_id_filepath[var_file.split(".")[0].split("_")[-1]] = var_file

In [172]:
dict_energy_id_filepath = {}
for var_file in (glob.glob("sundance/SunDance_data_release/energy/*")):
#     print(var_file.split(".")[0].split("_")[3])
    dict_energy_id_filepath[var_file.split(".")[0].split("_")[3]] = var_file

In [173]:
# #Format
print(dict_weather_id_filepath["10010"])
print(dict_energy_id_filepath["10010"])

sundance/SunDance_data_release/weather\SunDance_10010.csv
sundance/SunDance_data_release/energy\SunDance_10010_data.csv


In [174]:
# #Format:
# #{'id': ['sundance_weather_path.csv', 'sundance_energy_path.csv']
dict_weather_energy_id_filepath = {}

for key in (dict_weather_id_filepath.keys() | dict_energy_id_filepath.keys()):
    if key in dict_weather_id_filepath: dict_weather_energy_id_filepath.setdefault(key, []).append(dict_weather_id_filepath[key])
    if key in dict_energy_id_filepath: dict_weather_energy_id_filepath.setdefault(key, []).append(dict_energy_id_filepath[key])
        
# print(dict_weather_energy_id_filepath)

In [175]:
# #Test: Make sure that the files have have been mapped
for key, value in dict_weather_energy_id_filepath.items():
    if len(value) != 2:
        print(key, value)

This indicates that all the files in the weather and energy datasets have been mapped using the common identifier.

### Merge the Weather and Energy datasets based on Timestamp

In [176]:
header_sundance_weather = "dateyear,datemon,datemday,datehour,datemin,datetzname,tempm,tempi,dewptm,dewpti,hum,wspdm,\
wspdi,wgustm,wgusti,wdird,wdire,vism,visi,pressurem,pressurei,windchillm,windchilli,heatindexm,heatindexi,precipm,\
precipi,conds,icon,fog,rain,snow,hail,thunder,tornado,metar".split(",")

In [177]:
# #dict_weather_energy_id_filepath["10010"]
df_sundance_weather = pd.read_csv(dict_weather_energy_id_filepath["10010"][0], names= header_sundance_weather)
df_sundance_energy = pd.read_csv(dict_weather_energy_id_filepath["10010"][1])

In [178]:
df_sundance_weather.shape, df_sundance_energy.shape

((8556, 36), (8761, 5))

In [179]:
df_sundance_weather.head()

,dateyear,datemon,datemday,datehour,datemin,datetzname,tempm,tempi,dewptm,dewpti,...,precipi,conds,icon,fog,rain,snow,hail,thunder,tornado,metar
0,2015,1,1,0,16,America/Denver,-11.0,12.2,-16.0,3.2,...,-9999.0,Clear,clear,0.0,0.0,0,0,0.0,0.0,METAR KBDU 010716Z AUTO 19004KT 10SM CLR M11/M...
1,2015,1,1,1,16,America/Denver,-11.0,12.2,-16.0,3.2,...,-9999.0,Clear,clear,0.0,0.0,0,0,0.0,0.0,METAR KBDU 010816Z AUTO 00000KT 10SM CLR M11/M...
2,2015,1,1,2,16,America/Denver,-11.0,12.2,-17.0,1.4,...,-9999.0,Clear,clear,0.0,0.0,0,0,0.0,0.0,METAR KBDU 010916Z AUTO 00000KT 10SM CLR M11/M...
3,2015,1,1,3,16,America/Denver,-11.0,12.2,-18.0,-0.4,...,-9999.0,Clear,clear,0.0,0.0,0,0,0.0,0.0,METAR KBDU 011016Z AUTO 03003KT 10SM CLR M11/M...
4,2015,1,1,4,16,America/Denver,-12.0,10.4,-17.0,1.4,...,-9999.0,Clear,clear,0.0,0.0,0,0,0.0,0.0,METAR KBDU 011116Z AUTO 00000KT 10SM CLR M12/M...


In [180]:
df_sundance_energy.head(10)

,Date & Time,Grid,Solar,Load1,Load2
0,2016-01-01 05:00,12046.854780,29998.638518,1414.010673,-2035.617725
1,2016-01-01 04:00,12046.398447,29998.641113,1414.008903,-2035.451385
2,2016-01-01 03:00,12046.002883,29998.643668,1414.007119,-2035.288172
3,2016-01-01 02:00,12045.615644,29998.646296,1414.005360,-2035.129942
4,2016-01-01 01:00,12043.748957,29998.648868,1414.001672,-2034.929047
5,2016-01-01 00:00,12041.011377,29998.651456,1413.945631,-2034.746143
6,2015-12-31 23:00,12039.258004,29998.653860,1413.881442,-2034.600343
7,2015-12-31 22:00,12037.279656,29998.664177,1413.862536,-2034.422444
8,2015-12-31 21:00,12036.243378,29997.804695,1413.856587,-2034.349391
9,2015-12-31 20:00,12035.762090,29996.442008,1413.824808,-2034.333481


In [181]:
df_sundance_weather["datetime_yyyy_mm_dd_hh"] = df_sundance_weather.apply(
    lambda row: datetime(row['dateyear'], row['datemon'],row['datemday'], row['datehour']), axis=1)

df_sundance_energy["datetime_yyyy_mm_dd_hh"] = pd.to_datetime(df_sundance_energy["Date & Time"])

In [182]:
df_sundance_weather_join_energy = pd.merge(df_sundance_weather, df_sundance_energy, on="datetime_yyyy_mm_dd_hh")

In [183]:
df_sundance_weather_join_energy.shape

(8551, 42)

In [184]:
# #IN PROGRESS
# for var_file_id in list(dict_weather_energy_id_filepath.keys())[:5]:    
# #     print(var_file_id)

#     df_sundance_weather = pd.read_csv(dict_weather_energy_id_filepath[var_file_id][0], names= header_sundance_weather)
#     df_sundance_energy = pd.read_csv(dict_weather_energy_id_filepath[var_file_id][1])
#     print(df_sundance_weather.shape, df_sundance_energy.shape)
    
#     print(df_sundance_energy.head(20))